# Phenolopy Metrics module tests

## Globals

In [63]:
# set globals paths
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [64]:
# imports
import os
import random
import numpy as np
import xarray as xr
from IPython.utils import io

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import full arcpy toolbox
arcpy.ImportToolbox(r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

<module 'toolbox'>

### Reload libraries

In [65]:
# if scripts change, reload
from importlib import reload
reload(test_funcs)

<module 'test_funcs' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\tests\\code\\test_funcs.py'>

### Set data files and locations

In [22]:
# setup general io. nc ins and outs exist in these folders
input_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\phenolopy_metrics\inputs'
output_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\phenolopy_metrics\outputs'

# temp nc file for use when breaking ncs
temp_nc = os.path.join(input_folder, 'temp_nc.nc')  

# setup landsat cubes paths
ls_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_1_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc", # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_ls_90_20_raw_odc.nc", 
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\dwer_1_ls_90_20_raw_odc.nc",  
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_1_ls_10_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_3_ls_10_20_raw_odc.nc",   # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\tute_1_ls_10_20_raw_odc.nc",  # done all
]

# setup sentinel2 cubes paths
s2_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_s2_16_20_raw_odc.nc", 
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\dwer_1_s2_16_20_raw_odc.nc",  # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_2_s2_16_21_raw_odc.nc",   # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\tute_1_s2_18_20_raw_odc.nc",
]

### Set specific raw netcdf file

In [23]:
# set specific dataset
nc_file = ls_cubes[4]

### Set up function to iterate corruptor and tests

In [7]:
def run_corruptors_through_tests(in_nc, nc_coors, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed.
    """
    
    for nc_corr in nc_corrs:
        corr_name = nc_corr[0].__name__                   # name of current corruptor func
        corr_func, corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params
        
        # notify
        print('Corrupting NetCDF via: {}.\n'.format(corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            corr_func(**corr_params)
        except Exception as e:    
            print(e)
            print('Corruptor did not have enough data to work with. Skipping.\n')

        # iter each test func and apply to current corrupt nc
        for test in tests:
            test_nc_name = corr_name + '_' + test[0]    # name of current test nc
            test_func, test_params = test[1], test[2]   # pointer to test func and dict of params
            test_msg = test[3]
            
            # notify of test message
            print(test_msg)

            # create output nc file path and name and update params for in/out paths
            out_nc_file = os.path.join(output_folder, test_nc_name)
            
            # remove output nc if exists
            if os.path.exists(out_nc_file):
                os.remove(out_nc_file)
            
            # update params
            test_params.update({'in_nc': temp_nc, 'out_nc': out_nc_file})

            # perform current test
            try:
                # notify
                print('Performing test: {}.'.format(test_nc_name))

                # perform test, provide prints if requested
                if not verbose:
                    with io.capture_output() as cap:
                        test_func(**test_params)
                else:
                    test_func(**test_params)
                    print('\n')

            except Exception as e:    
                print(e)

        # notify
        print('All tests applied to corruptor NetCDF.\n\n')

### Set up netcdf corruptor functions

In [24]:
# these are numerous netcdf corruptors. feed a raw nc in, break it, output as temp nc
# comment out any that are irrelevant
# each of these uncommented will be fed through the tests below
def build_nc_corruptors(temp_nc):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    cs = []

    # func: raw default dataset, no changes
    cs.append([test_funcs.nc_default, {'in_nc': temp_nc}])

    # func: remove x, y, time, spatial_ref coords
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'x'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'y'}])
    cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'time'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'spatial_ref'}])

    #func: remove red and oa_fmask band vars
    #cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'nbart_red'}])
    cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'oa_fmask'}])

    #func: limit number of years in various combos
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2011, 'e_year': 2013}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2019, 'e_year': 2020}])
    cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2018, 'e_year': 2018}])

    #func: set all vars to nan
    cs.append([test_funcs.set_nc_vars_all_nan, {'in_nc': temp_nc}])

    #func: set all vars to zero
    cs.append([test_funcs.set_nc_vars_all_zero, {'in_nc': temp_nc}])

    #func: set all vars for 10 rand times to all nan
    cs.append([test_funcs.set_nc_vars_random_all_nan, {'in_nc': temp_nc, 'num': 10}])

    #func: strip all attrs from nc    
    cs.append([test_funcs.strip_nc_attributes, {'in_nc': temp_nc}])

    #func: set vars in first and last time index to all nan
    cs.append([test_funcs.set_end_times_to_all_nan, {'in_nc': temp_nc}])

    #func: reduce whole nc to one random time slice
    #cs.append([test_funcs.reduce_to_one_scene, {'in_nc': temp_nc}])

    #func: set wet months all nan, all years, for specific months
    cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]}])

    #func: drop wet months, all years, for specific months
    cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]}])

    #func: remove crs attribute
    cs.append([test_funcs.remove_crs_attr, {'in_nc': temp_nc}])

    #func: remove nodatavals attribute
    #cs.append([test_funcs.remove_nodatavals_attr, {'in_nc': temp_nc}])
    
    return cs

nc_corruptors = build_nc_corruptors(temp_nc=temp_nc)

## Run tests

### Test One: Use all dates

In [25]:
def build_test_one_funcs(in_nc, temp_nc):
    """sets up test one functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': '2019;2020',         # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
    
    msg = 'Running Test One: default in use all dates.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': True})
    ts.append(['t_1_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test One: use all dates is false.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': False})
    ts.append(['t_1_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    

    msg = 'Running Test One: use all dates is None (default to True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': None})
    ts.append(['t_1_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])   
            
    return ts

### Test One: Run!

In [27]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_one_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test One: default in use all dates.
Performing test: nc_default_t_1_def.nc.
Running Test One: use all dates is false.
Performing test: nc_default_t_1_a.nc.
Running Test One: use all dates is None (default to True).
Performing test: nc_default_t_1_b.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test One: default in use all dates.
Performing test: remove_coord_t_1_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test One: use all dates is false.
Performing test: remove_coord_t_1_a.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test One: use all dates is None (default to True).
Performing test: remove_coord_t_1_b.nc.
In

### Test Two: Specific years

In [28]:
def build_test_two_funcs(in_nc, temp_nc):
    """sets up test two functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': False,                # use all dates
        'in_specific_years': '2020',              # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test Two: default specific year (2020).'
    params = inputs.copy()
    params.update({'in_specific_years': '2020'})
    ts.append(['t_2_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Two: latest three years (2018, 2019, 2020).'
    params = inputs.copy()
    params.update({'in_specific_years': '2018;2019;2020'})
    ts.append(['t_2_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    rand = str(random.randint(1990, 2020))
    msg = 'Running Test Two: single random year ({}).'.format(rand)
    params = inputs.copy()
    params.update({'in_specific_years': rand})
    ts.append(['t_2_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    

    msg = 'Running Test Two: bad year (2012).'
    params = inputs.copy()
    params.update({'in_specific_years': '2012'})
    ts.append(['t_2_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    
    
    msg = 'Running Test Two: multi years (2010-2020).'
    params = inputs.copy()
    params.update({'in_specific_years': '2010;2011;2012;2013;2014;2015;2016;2017;2018;2019;2020'})
    ts.append(['t_2_d.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 

    msg = 'Running Test Two: years 2019, 2020 selected but use all set to true.'
    params = inputs.copy()
    params.update({'in_specific_years': '2019;2020', 'in_use_all_dates': True})
    ts.append(['t_2_e.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    msg = 'Running Test Two: years set to none (default).'
    params = inputs.copy()
    params.update({'in_specific_years': None})
    ts.append(['t_2_f.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    msg = 'Running Test Two: years set to empty "".'
    params = inputs.copy()
    params.update({'in_specific_years': ''})
    ts.append(['t_2_g.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    msg = 'Running Test Two: years is wrong (30).'
    params = inputs.copy()
    params.update({'in_specific_years': 30})
    ts.append(['t_2_h.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
        
    return ts

### Test Two: Run!

In [29]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_two_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Two: default specific year (2020).
Performing test: nc_default_t_2_def.nc.
Running Test Two: latest three years (2018, 2019, 2020).
Performing test: nc_default_t_2_a.nc.
Running Test Two: single random year (2008).
Performing test: nc_default_t_2_b.nc.
Running Test Two: bad year (2012).
Performing test: nc_default_t_2_c.nc.
Cannot use years 2011 or 2012, insufficient data.
Failed to execute (Phenolopy_Metrics).

Running Test Two: multi years (2010-2020).
Performing test: nc_default_t_2_d.nc.
Cannot use years 2011 or 2012, insufficient data.
Failed to execute (Phenolopy_Metrics).

Running Test Two: years 2019, 2020 selected but use all set to true.
Performing test: nc_default_t_2_e.nc.
Running Test Two: years set to none (default).
Performing test: nc_default_t_2_f.nc.
Did not provide a specific year(s).
Failed to execute (Phenolopy_Met

Corrupting NetCDF via: set_nc_vars_all_zero.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Setting nc var data to all zero
Running Test Two: default specific year (2020).
Performing test: set_nc_vars_all_zero_t_2_def.nc.
NetCDF has empty variables. Please download again.
Failed to execute (Phenolopy_Metrics).

Running Test Two: latest three years (2018, 2019, 2020).
Performing test: set_nc_vars_all_zero_t_2_a.nc.
NetCDF has empty variables. Please download again.
Failed to execute (Phenolopy_Metrics).

Running Test Two: single random year (2008).
Performing test: set_nc_vars_all_zero_t_2_b.nc.
NetCDF has empty variables. Please download again.
Failed to execute (Phenolopy_Metrics).

Running Test Two: bad year (2012).
Performing test: set_nc_vars_all_zero_t_2_c.nc.
NetCDF has empty variables. Please download again.
Failed to execute (Phenolopy_Metrics).

Running Test Two: multi years (2010-2020).
Performing test: set_nc_vars_all_zero_t_2_d.nc.
NetCDF has empty variables. 

### Test Three: Vegetation Idx

In [30]:
def build_test_three_funcs(in_nc, temp_nc):
    """sets up test three functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                # use all dates
        'in_specific_years': '2020',              # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test Three: default veg idx (mavi).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MAVI'})
    ts.append(['t_3_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Three: veg idx is "".'
    params = inputs.copy()
    params.update({'in_veg_idx': ''})
    ts.append(['t_3_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Three: veg idx is NDVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NDVI'})
    ts.append(['t_3_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Three: veg idx is EVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'EVI'})
    ts.append(['t_3_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Three: veg idx is SAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'SAVI'})
    ts.append(['t_3_d.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Three: veg idx is MSAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MSAVI'})
    ts.append(['t_3_e.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test Three: veg idx is SLAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'SLAVI'})
    ts.append(['t_3_f.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test Three: veg idx is MAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MAVI'})
    ts.append(['t_3_g.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Three: veg idx is kNDVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'kNDVI'})
    ts.append(['t_3_h.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    

    msg = 'Running Test Three: veg idx is NRVA (non-existant).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NRVA'})
    ts.append(['t_3_i.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    msg = 'Running Test Three: veg idx is TCG (not supported).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NRVA'})
    ts.append(['t_3_j.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    
        
    return ts

### Test Three: Run!

In [31]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_three_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Three: default veg idx (mavi).
Performing test: nc_default_t_3_def.nc.
Running Test Three: veg idx is "".
Performing test: nc_default_t_3_a.nc.
Running Test Three: veg idx is NDVI.
Performing test: nc_default_t_3_b.nc.
Running Test Three: veg idx is EVI.
Performing test: nc_default_t_3_c.nc.
Running Test Three: veg idx is SAVI.
Performing test: nc_default_t_3_d.nc.
Running Test Three: veg idx is MSAVI.
Performing test: nc_default_t_3_e.nc.
Running Test Three: veg idx is SLAVI.
Performing test: nc_default_t_3_f.nc.
Running Test Three: veg idx is MAVI.
Performing test: nc_default_t_3_g.nc.
Running Test Three: veg idx is kNDVI.
Performing test: nc_default_t_3_h.nc.
Running Test Three: veg idx is NRVA (non-existant).
Performing test: nc_default_t_3_i.nc.
Failed to execute. Parameters are not valid.
ERROR 000800: The value is not a member o

Performing test: set_end_times_to_all_nan_t_3_d.nc.
Running Test Three: veg idx is MSAVI.
Performing test: set_end_times_to_all_nan_t_3_e.nc.
Running Test Three: veg idx is SLAVI.
Performing test: set_end_times_to_all_nan_t_3_f.nc.
Running Test Three: veg idx is MAVI.
Performing test: set_end_times_to_all_nan_t_3_g.nc.
Running Test Three: veg idx is kNDVI.
Performing test: set_end_times_to_all_nan_t_3_h.nc.
Running Test Three: veg idx is NRVA (non-existant).
Performing test: set_end_times_to_all_nan_t_3_i.nc.
Failed to execute. Parameters are not valid.
ERROR 000800: The value is not a member of NDVI | EVI | SAVI | MSAVI | SLAVI | MAVI | kNDVI.
Failed to execute (Phenolopy_Metrics).

Running Test Three: veg idx is TCG (not supported).
Performing test: set_end_times_to_all_nan_t_3_j.nc.
Failed to execute. Parameters are not valid.
ERROR 000800: The value is not a member of NDVI | EVI | SAVI | MSAVI | SLAVI | MAVI | kNDVI.
Failed to execute (Phenolopy_Metrics).

All tests applied to corr

### Test Four: Metrics

In [32]:
def build_test_four_funcs(in_nc, temp_nc):
    """sets up test four functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                # use all dates
        'in_specific_years': '2020',              # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    metrics = np.array([
        "'POS: Peak of season'",
        "'VOS: Valley of season'",
        "'BSE: Base of season'",
        "'MOS: Middle of season'",
        "'AOS: Amplitude of season'",
        "'SOS: Start of season'",
        "'EOS: End of season'",
        "'LOS: Length of season'",
        "'ROI: Rate of increase'",
        "'ROD: Rate of decrease'",
        "'SIOS: Short integral of season'",
        "'LIOS: Long integral of season'",
        "'SIOT: Short integral of total'",
        "'LIOT: Long integral of total'",
        "'NOS: Number of seasons'"
    ])
    
    ts = []

    msg = 'Running Test Four: default metrics (pos, vos, sos, eos, roi, rod, los, sio, lio).'
    params = inputs.copy()
    params.update({'in_metrics': ";".join(metrics[[0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13]])})
    ts.append(['t_4_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Four: metrics is None (will do default).'
    params = inputs.copy()
    params.update({'in_metrics': None})
    ts.append(['t_4_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Four: lios only.'
    params = inputs.copy()
    params.update({'in_metrics': ';'.join(metrics[[11]])})
    ts.append(['t_4_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Four: only non-default bse, mos, aos, nos.'
    params = inputs.copy()
    params.update({'in_metrics': ';'.join(metrics[[2, 3, 4, 14]])})
    ts.append(['t_4_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])   
    
    msg = 'Running Test Four: input is "".'
    params = inputs.copy()
    params.update({'in_metrics': ""})
    ts.append(['t_4_d.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])       
      
    return ts

### Test Four: Run!

In [33]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_four_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Four: default metrics (pos, vos, sos, eos, roi, rod, los, sio, lio).
Performing test: nc_default_t_4_def.nc.
Running Test Four: metrics is None (will do default).
Performing test: nc_default_t_4_a.nc.
Running Test Four: lios only.
Performing test: nc_default_t_4_b.nc.
Running Test Four: only non-default bse, mos, aos, nos.
Performing test: nc_default_t_4_c.nc.
Running Test Four: input is "".
Performing test: nc_default_t_4_d.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Phenological metrics: Value is required
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test Four: default metrics (pos, vos, sos, eos, roi, rod, los, sio, lio).
Performing test: remove_coord_t_4


All tests applied to corruptor NetCDF.


Corrupting NetCDF via: set_end_times_to_all_nan.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Setting end times to all nan
Running Test Four: default metrics (pos, vos, sos, eos, roi, rod, los, sio, lio).
Performing test: set_end_times_to_all_nan_t_4_def.nc.
Running Test Four: metrics is None (will do default).
Performing test: set_end_times_to_all_nan_t_4_a.nc.
Running Test Four: lios only.
Performing test: set_end_times_to_all_nan_t_4_b.nc.
Running Test Four: only non-default bse, mos, aos, nos.
Performing test: set_end_times_to_all_nan_t_4_c.nc.
Running Test Four: input is "".
Performing test: set_end_times_to_all_nan_t_4_d.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Phenological metrics: Value is required
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: set_all_specific_season_nan.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Setting al

### Test Five: Method type

In [34]:
def build_test_five_funcs(in_nc, temp_nc):
    """sets up test five functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                # use all dates
        'in_specific_years': '2020',              # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test Five: default method (seasonal amp).'
    params = inputs.copy()
    params.update({'in_method_type': 'Seasonal amplitude'})
    ts.append(['t_5_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test Five: absolute amp method'
    params = inputs.copy()
    params.update({'in_method_type': 'Absolute amplitude'})
    ts.append(['t_5_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    

    msg = 'Running Test Five: relative amp method'
    params = inputs.copy()
    params.update({'in_method_type': 'Relative amplitude'})
    ts.append(['t_5_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])     
    
    msg = 'Running Test Five: first of slope method'
    params = inputs.copy()
    params.update({'in_method_type': 'First of slope'})
    ts.append(['t_5_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    msg = 'Running Test Five: mean of slope method'
    params = inputs.copy()
    params.update({'in_method_type': 'Mean of slope'})
    ts.append(['t_5_d.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    msg = 'Running Test Five: method doesnt exist (derp slope)'
    params = inputs.copy()
    params.update({'in_method_type': 'derp sloper'})
    ts.append(['t_5_e.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
                
    return ts

### Test Five: Run!

In [35]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_five_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Five: default method (seasonal amp).
Performing test: nc_default_t_5_def.nc.
Running Test Five: absolute amp method
Performing test: nc_default_t_5_a.nc.
Running Test Five: relative amp method
Performing test: nc_default_t_5_b.nc.
Running Test Five: first of slope method
Performing test: nc_default_t_5_c.nc.
Running Test Five: mean of slope method
Performing test: nc_default_t_5_d.nc.
Running Test Five: method doesnt exist (derp slope)
Performing test: nc_default_t_5_e.nc.
Failed to execute. Parameters are not valid.
ERROR 000800: The value is not a member of First of slope | Mean of slope | Seasonal amplitude | Absolute amplitude | Relative amplitude.
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removin

### Test Six: Amplitude factor

In [36]:
def build_test_six_funcs(in_nc, temp_nc):
    """sets up test six functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                # use all dates
        'in_specific_years': '2020',              # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test six: default seasonal amp factor (0.75).'
    params = inputs.copy()
    params.update({'in_method_type': 'Seasonal amplitude', 'in_amp_factor': 0.75})
    ts.append(['t_6_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test six: seasonal amp factor (0.1).'
    params = inputs.copy()
    params.update({'in_method_type': 'Seasonal amplitude', 'in_amp_factor': 0.1})
    ts.append(['t_6_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test six: seasonal amp factor (1).'
    params = inputs.copy()
    params.update({'in_method_type': 'Seasonal amplitude', 'in_amp_factor': 1})
    ts.append(['t_6_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test six: relative amp factor (0.3).'
    params = inputs.copy()
    params.update({'in_method_type': 'Relative amplitude', 'in_amp_factor': 0.3})
    ts.append(['t_6_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test six: relative amp factor (0.98).'
    params = inputs.copy()
    params.update({'in_method_type': 'Relative amplitude', 'in_amp_factor': 0.98})
    ts.append(['t_6_d.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test six: seasonal amp factor unspported (99.'
    params = inputs.copy()
    params.update({'in_method_type': 'Relative amplitude', 'in_amp_factor': 99})
    ts.append(['t_6_d.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    return ts

### Test Six: Run!

In [37]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_six_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test six: default seasonal amp factor (0.75).
Performing test: nc_default_t_6_def.nc.
Running Test six: seasonal amp factor (0.1).
Performing test: nc_default_t_6_a.nc.
Running Test six: seasonal amp factor (1).
Performing test: nc_default_t_6_b.nc.
Running Test six: relative amp factor (0.3).
Performing test: nc_default_t_6_c.nc.
Running Test six: relative amp factor (0.98).
Performing test: nc_default_t_6_d.nc.
Running Test six: seasonal amp factor unspported (99.
Performing test: nc_default_t_6_d.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 0 to 1.
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test six: default seasonal amp fa

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Stripping attributes from nc dim
Running Test six: default seasonal amp factor (0.75).
Performing test: strip_nc_attributes_t_6_def.nc.
NetCDF must have attributes.
Failed to execute (Phenolopy_Metrics).

Running Test six: seasonal amp factor (0.1).
Performing test: strip_nc_attributes_t_6_a.nc.
NetCDF must have attributes.
Failed to execute (Phenolopy_Metrics).

Running Test six: seasonal amp factor (1).
Performing test: strip_nc_attributes_t_6_b.nc.
NetCDF must have attributes.
Failed to execute (Phenolopy_Metrics).

Running Test six: relative amp factor (0.3).
Performing test: strip_nc_attributes_t_6_c.nc.
NetCDF must have attributes.
Failed to execute (Phenolopy_Metrics).

Running Test six: relative amp factor (0.98).
Performing test: strip_nc_attributes_t_6_d.nc.
NetCDF must have attributes.
Failed to execute (Phenolopy_Metrics).

Running Test six: seasonal amp factor unspported (99.
Performing test: strip_nc_attributes_

### Test Seven: Absolute value

In [38]:
def build_test_seven_funcs(in_nc, temp_nc):
    """sets up test seven functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': '2020',              # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Absolute amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test seven: default absolute value (0.3).'
    params = inputs.copy()
    params.update({'in_method_type': 'Absolute amplitude', 'in_abs_value': 0.3})
    ts.append(['t_7_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test seven: absolute value (1.2).'
    params = inputs.copy()
    params.update({'in_method_type': 'Absolute amplitude', 'in_abs_value': 1.2})
    ts.append(['t_7_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test seven: absolute value (-0.8).'
    params = inputs.copy()
    params.update({'in_method_type': 'Absolute amplitude', 'in_abs_value': -0.8})
    ts.append(['t_7_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Seven: Run!

In [39]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_seven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test seven: default absolute value (0.3).
Performing test: nc_default_t_7_def.nc.
Running Test seven: absolute value (1.2).
Performing test: nc_default_t_7_a.nc.
Running Test seven: absolute value (-0.8).
Performing test: nc_default_t_7_b.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test seven: default absolute value (0.3).
Performing test: remove_coord_t_7_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test seven: absolute value (1.2).
Performing test: remove_coord_t_7_a.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test seven: absolute value (-0.8).
Performing test: remove_coord_t_7_b.nc.
Input NetCDF must have

### Test Eight: Savitsky window length

In [40]:
def build_test_eight_funcs(in_nc, temp_nc):
    """sets up test eight functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test eight: default savitsky win length (3).'
    params = inputs.copy()
    params.update({'in_sav_win_length': 3})
    ts.append(['t_8_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test eight: savitsky win length even num (6) - func will change to 3.'
    params = inputs.copy()
    params.update({'in_sav_win_length': 6})
    ts.append(['t_8_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test eight: savitsky win length very large (99).'
    params = inputs.copy()
    params.update({'in_sav_win_length': 99})
    ts.append(['t_8_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test eight: savitsky win length  None.'
    params = inputs.copy()
    params.update({'in_sav_win_length': None})
    ts.append(['t_8_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Eight: Run!

In [41]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eight_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test eight: default savitsky win length (3).
Performing test: nc_default_t_8_def.nc.
Running Test eight: savitsky win length even num (6) - func will change to 3.
Performing test: nc_default_t_8_a.nc.
Running Test eight: savitsky win length very large (99).
Performing test: nc_default_t_8_b.nc.
Running Test eight: savitsky win length  None.
Performing test: nc_default_t_8_c.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test eight: default savitsky win length (3).
Performing test: remove_coord_t_8_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test eight: savitsky win length even num (6) - func will change to 3.
Performing test: remove_coord_t_8_a.nc.
Input NetCDF 

### Test Nine: Savitsky polyorder

In [42]:
def build_test_nine_funcs(in_nc, temp_nc):
    """sets up test nine functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test nine: default savitsky polyorder (1)).'
    params = inputs.copy()
    params.update({'in_sav_polyorder': 3})
    ts.append(['t_9_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test nine: savitsky polyorder higher than win length (5).'
    params = inputs.copy()
    params.update({'in_sav_polyorder': 5})
    ts.append(['t_9_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test nine: savitsky polyorder is 13 while win length is 51.'
    params = inputs.copy()
    params.update({'in_sav_polyorder': 13, 'in_sav_win_length': 51})
    ts.append(['t_9_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test nine: savitsky polyorder is 0.'
    params = inputs.copy()
    params.update({'in_sav_polyorder': 0})
    ts.append(['t_9_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Ten: Run!

In [43]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_nine_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test nine: default savitsky polyorder (1)).
Performing test: nc_default_t_9_def.nc.
Running Test nine: savitsky polyorder higher than win length (5).
Performing test: nc_default_t_9_a.nc.
Running Test nine: savitsky polyorder is 13 while win length is 51.
Performing test: nc_default_t_9_b.nc.
Running Test nine: savitsky polyorder is 0.
Performing test: nc_default_t_9_c.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 1 to 100.
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test nine: default savitsky polyorder (1)).
Performing test: remove_coord_t_9_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metr

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Setting all season to nan with months: [7, 8, 9, 10, 11, 12]
Running Test nine: default savitsky polyorder (1)).
Performing test: set_all_specific_season_nan_t_9_def.nc.
Running Test nine: savitsky polyorder higher than win length (5).
Performing test: set_all_specific_season_nan_t_9_a.nc.
Running Test nine: savitsky polyorder is 13 while win length is 51.
Performing test: set_all_specific_season_nan_t_9_b.nc.
Running Test nine: savitsky polyorder is 0.
Performing test: set_all_specific_season_nan_t_9_c.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 1 to 100.
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_all_specific_season_nan.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Remove all season with months: [7, 8, 9, 10, 11, 12]
Running Test nine: default savitsky polyorder (1)).
Performing test: rem

### Test Ten: Outlier window length

In [44]:
def build_test_ten_funcs(in_nc, temp_nc):
    """sets up test ten functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test ten: default outlier win length (3).'
    params = inputs.copy()
    params.update({'in_out_win_length': 3})
    ts.append(['t_10_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test ten: outlier win length even num (6) - func will change to 3.'
    params = inputs.copy()
    params.update({'in_out_win_length': 6})
    ts.append(['t_10_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test ten: outlier win length very large (99).'
    params = inputs.copy()
    params.update({'in_out_win_length': 99})
    ts.append(['t_10_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test ten: outlier win length  None.'
    params = inputs.copy()
    params.update({'in_out_win_length': None})
    ts.append(['t_10_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Ten: Run!

In [45]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_ten_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test ten: default outlier win length (3).
Performing test: nc_default_t_10_def.nc.
Running Test ten: outlier win length even num (6) - func will change to 3.
Performing test: nc_default_t_10_a.nc.
Running Test ten: outlier win length very large (99).
Performing test: nc_default_t_10_b.nc.
Running Test ten: outlier win length  None.
Performing test: nc_default_t_10_c.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test ten: default outlier win length (3).
Performing test: remove_coord_t_10_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test ten: outlier win length even num (6) - func will change to 3.
Performing test: remove_coord_t_10_a.nc.
Input NetCDF must have x,

### Test Eleven: Outlier factor

In [46]:
def build_test_eleven_funcs(in_nc, temp_nc):
    """sets up test eleven functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test eleven: default outlier factor (2).'
    params = inputs.copy()
    params.update({'in_out_factor': 3})
    ts.append(['t_eleven_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test eleven: outlier factor very high (30).'
    params = inputs.copy()
    params.update({'in_out_factor': 30})
    ts.append(['t_eleven_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test eleven: outlier factor 0.'
    params = inputs.copy()
    params.update({'in_out_factor': 0, 'in_sav_win_length': 51})
    ts.append(['t_eleven_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Eleven: Run!

In [47]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eleven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test eleven: default outlier factor (2).
Performing test: nc_default_t_eleven_def.nc.
Running Test eleven: outlier factor very high (30).
Performing test: nc_default_t_eleven_a.nc.
Running Test eleven: outlier factor 0.
Performing test: nc_default_t_eleven_b.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 1 to 100.
Failed to execute (Phenolopy_Metrics).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test eleven: default outlier factor (2).
Performing test: remove_coord_t_eleven_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test eleven: outlier factor very high (30).
Performing test: remove_coord_t_eleven_a.nc.
Input N

### Test Twelve: Outlier factor

In [48]:
def build_test_twelve_funcs(in_nc, temp_nc):
    """sets up test twelve functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test twelve: default fix edges (True).'
    params = inputs.copy()
    params.update({'in_fix_edges': True})
    ts.append(['t_twelve_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test twelve: fix edges False.'
    params = inputs.copy()
    params.update({'in_fix_edges': False})
    ts.append(['t_twelve_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test twelve: fix edges False and fill nan is True.'
    params = inputs.copy()
    params.update({'in_fix_edges': False, 'in_fill_nans': True})
    ts.append(['t_twelve_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Twelve: Run!

In [49]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_twelve_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test twelve: default fix edges (True).
Performing test: nc_default_t_twelve_def.nc.
Running Test twelve: fix edges False.
Performing test: nc_default_t_twelve_a.nc.
Running Test twelve: fix edges False and fill nan is True.
Performing test: nc_default_t_twelve_b.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test twelve: default fix edges (True).
Performing test: remove_coord_t_twelve_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test twelve: fix edges False.
Performing test: remove_coord_t_twelve_a.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test twelve: fix edges False and fill nan is True.
Performing test: re

### Test Thirteen: Fix edges

In [50]:
def build_test_thirteen_funcs(in_nc, temp_nc):
    """sets up test thirteen functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test thirteen: default fill nans (True).'
    params = inputs.copy()
    params.update({'in_fill_nans': True})
    ts.append(['t_thirteen_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test thirteen: fill nans False.'
    params = inputs.copy()
    params.update({'in_fill_nans': False})
    ts.append(['t_thirteen_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test thirteen: fix edges False and fill nan is False.'
    params = inputs.copy()
    params.update({'in_fill_nans': False, 'in_fill_nans': True})
    ts.append(['t_thirteen_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    return ts

### Test Thirteen: Run!

In [51]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_thirteen_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test thirteen: default fill nans (True).
Performing test: nc_default_t_thirteen_def.nc.
Running Test thirteen: fill nans False.
Performing test: nc_default_t_thirteen_a.nc.
Running Test thirteen: fix edges False and fill nan is False.
Performing test: nc_default_t_thirteen_b.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test thirteen: default fill nans (True).
Performing test: remove_coord_t_thirteen_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test thirteen: fill nans False.
Performing test: remove_coord_t_thirteen_a.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test thirteen: fix edges False and fill nan is Fa

### Test Fourteen: Pixel Flags

In [52]:
def build_test_fourteen_funcs(in_nc, temp_nc):
    """sets up test fourteen functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test Fourteen: pixel flags Valid;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid;Snow;Water'})
    ts.append(['t_14_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Fourteen: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'NoData;Valid;Cloud;Shadow;Snow;Water'})
    ts.append(['t_14_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    

    msg = 'Running Test Fourteen: pixel flags is Valid.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid'})
    ts.append(['t_14_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 

    msg = 'Running Test Fourteen: pixel flags is Cloud;Shadow.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Cloud;Shadow'})
    ts.append(['t_14_c.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg]) 
    
    return ts

### Test Fourteen: Run!

In [53]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_fourteen_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Fourteen: pixel flags Valid;Snow;Water.
Performing test: nc_default_t_14_def.nc.
Running Test Fourteen: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.
Performing test: nc_default_t_14_a.nc.
Running Test Fourteen: pixel flags is Valid.
Performing test: nc_default_t_14_b.nc.
Running Test Fourteen: pixel flags is Cloud;Shadow.
Performing test: nc_default_t_14_c.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test Fourteen: pixel flags Valid;Snow;Water.
Performing test: remove_coord_t_14_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test Fourteen: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.
Performing test: remove_coord_t_14_a.nc.
Input NetCDF must 

### Test Fifteen: Max Cloud Cover

In [54]:
def build_test_fifteen_funcs(in_nc, temp_nc):
    """sets up test fifteen functions"""
    
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': None,                # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                   # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                   # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test Fifteen: default max cloud (10).'
    params = inputs.copy()
    params.update({'in_max_cloud': 10})
    ts.append(['t_15_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])

    msg = 'Running Test Fifteen: max cloud cover is 0.'
    params = inputs.copy()
    params.update({'in_max_cloud': 0})
    ts.append(['t_15_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Fifteen: max cloud cover is 50.'
    params = inputs.copy()
    params.update({'in_max_cloud': 90})
    ts.append(['t_15_b.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])    
    
    return ts

### Test Fifteen: Run!

In [55]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_fifteen_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Fifteen: default max cloud (10).
Performing test: nc_default_t_15_def.nc.
Running Test Fifteen: max cloud cover is 0.
Performing test: nc_default_t_15_a.nc.
Running Test Fifteen: max cloud cover is 50.
Performing test: nc_default_t_15_b.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test Fifteen: default max cloud (10).
Performing test: remove_coord_t_15_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test Fifteen: max cloud cover is 0.
Performing test: remove_coord_t_15_a.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (Phenolopy_Metrics).

Running Test Fifteen: max cloud cover is 50.
Performing test: remove_coord_t_15_b.nc.
Input NetCDF mus

### Test Sixteen: Add To Map

In [56]:
def build_test_sixteen_funcs(in_nc, temp_nc):
    """sets up test sixteen functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_aggregate': False,                    # aggregate output
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': '',             # add result to map
    }
    
    # set up list
    ts = []
            
    msg = 'Running Test Sixteen: default add result to map (True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': True})
    ts.append(['t_16_def.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    msg = 'Running Test Sixteen: add result to map is False.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': False})
    ts.append(['t_16_a.nc', arcpy.Phenolopy_Metrics_toolbox , params, msg])
    
    return ts

### Test Sixteen: Run!

In [57]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_sixteen_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Sixteen: default add result to map (True).
Performing test: nc_default_t_16_def.nc.
Phenolopy_Metrics() got an unexpected keyword argument 'in_nc_file'
Running Test Sixteen: add result to map is False.
Performing test: nc_default_t_16_a.nc.
Phenolopy_Metrics() got an unexpected keyword argument 'in_nc_file'
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test Sixteen: default add result to map (True).
Performing test: remove_coord_t_16_def.nc.
Phenolopy_Metrics() got an unexpected keyword argument 'in_nc_file'
Running Test Sixteen: add result to map is False.
Performing test: remove_coord_t_16_a.nc.
Phenolopy_Metrics() got an unexpected keyword argument 'in_nc_file'
All tests applied to corruptor NetCDF.


Corrupting 

# Working

In [ ]:
# resample 1w
# interp
# subset to specific year(s)
# group by week (creates 52 weeks)
# compute
# remove outliers
# resample to 1w, 0.5m, 1 m
# remove overshoot dates
# smooth
# do pheno
# do nos

In [ ]:
# consider dask all 1, instead of -1
# consider using resample().interpolate(linear) instead of resample.median then interpolate
# do neither of these... 

In [ ]:
%matplotlib inline

import time
import xarray as xr
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# shared folder
sys.path.append(FOLDER_SHARED)
import arc, satfetcher, tools

# import testing functions
sys.path.append(FOLDER_MODULES)
import phenolopy

# if scripts change, reload
from importlib import reload
reload(phenolopy)

In [ ]:
years = [2018, 2019, 2020]

i = 50
fix_edges = True

# load prepared ds
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\test\ls_tute.nc")
#ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\test\s2_tute.nc")
ds = ds.chunk({'time': -1})
#ds = ds.load()
ds = ds.isel(x=slice(0, 150), y=slice(0, 150))

# check if current year selected
from datetime import datetime
if years is not None:
    if datetime.now().year in years:
        raise ValueError('Can not use current year.')
        
# check if 2011, 2012 ind ataset, error if so
if 2011 in years or 2012 in years:
    raise ValueError('Can not use years 2011 or 2012.')
        

# ensure edge dates exist for adequate interpolation
# if years is none, gets min, max years in ds
ds = phenolopy.enforce_edge_dates(ds=ds, years=years)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='red')
plt.show()


# subset with buffer dates either side, if years none, returns orig ds
ds = phenolopy.subset_via_years_with_buffers(ds=ds, years=years)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='orange')
plt.show()


# resample to fortnights
ds = phenolopy.resample(ds=ds, interval='SMS')
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='blue')
plt.show()


# interp any gaps from resample
ds = phenolopy.interpolate(ds=ds)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='green')
plt.show()


# set outliers to nan from fortnight
ds = phenolopy.remove_spikes(ds=ds, user_factor=2, win_size=3)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='aqua')
plt.show()


# interpolate once more to fill outlier gaps
ds = phenolopy.interpolate(ds=ds)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='purple')
plt.show()


# smooth the dataset
ds = phenolopy.smooth(ds=ds, var='veg_idx', window_length=3, polyorder=1)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='darkblue')
plt.show()


# do final subset to ensure only requested years in dataset
# if years is none, returns original ds
ds = phenolopy.subset_via_years(ds=ds, years=years)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='black')
plt.show()


# if len(ds['time']) not equal number (fortnight 12, 24, 48, etc)
# todo

# group years by m-d (1-1, 1-15, 2-1, 2-15, etc)
ds = phenolopy.group(ds=ds)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='gold', marker='o')
plt.show()


# do final resample up to to 365 days
ds = phenolopy.resample(ds=ds, interval='1D')
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='maroon', marker='o')
plt.show()


# interpolate our data one last time to fill those 365 days in
ds = phenolopy.interpolate(ds=ds)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='darkgrey', marker='o')
plt.show()


# do safety check to ensure no overshoot dates
ds = phenolopy.drop_overshoot_dates(ds=ds, min_dates=3)
fig = plt.figure(figsize=[15, 2])
ds['veg_idx'].isel(x=i, y=i).plot(color='grey', marker='o')
plt.show()


# if len(ds['time']) != 365: fail? or warn?
#

# run phenolopy

In [ ]:
ds = ds.compute()

In [ ]:
#if peak is 365
# get next highest peak
# select next closest

In [ ]:
        # var = 'pos_times'
        # ds[var] = xr.where((ds[var] > 0)   & (ds[var] <= 31),  1,  ds[var])
        # ds[var] = xr.where((ds[var] > 31)  & (ds[var] <= 59),  2,  ds[var])
        # ds[var] = xr.where((ds[var] > 59)  & (ds[var] <= 90),  3,  ds[var])
        # ds[var] = xr.where((ds[var] > 90)  & (ds[var] <= 120), 4,  ds[var])
        # ds[var] = xr.where((ds[var] > 120) & (ds[var] <= 151), 5,  ds[var])
        # ds[var] = xr.where((ds[var] > 151) & (ds[var] <= 181), 6,  ds[var])
        # ds[var] = xr.where((ds[var] > 181) & (ds[var] <= 212), 7,  ds[var])
        # ds[var] = xr.where((ds[var] > 212) & (ds[var] <= 243), 8,  ds[var])
        # ds[var] = xr.where((ds[var] > 243) & (ds[var] <= 273), 9,  ds[var])
        # ds[var] = xr.where((ds[var] > 273) & (ds[var] <= 304), 10, ds[var])
        # ds[var] = xr.where((ds[var] > 304) & (ds[var] <= 334), 11, ds[var])
        # ds[var] = xr.where((ds[var] > 334) & (ds[var] <= 365), 12, ds[var])
        
        # var = 'vos_times'
        # ds[var] = xr.where((ds[var] > 0)   & (ds[var] <= 31),  1,  ds[var])
        # ds[var] = xr.where((ds[var] > 31)  & (ds[var] <= 59),  2,  ds[var])
        # ds[var] = xr.where((ds[var] > 59)  & (ds[var] <= 90),  3,  ds[var])
        # ds[var] = xr.where((ds[var] > 90)  & (ds[var] <= 120), 4,  ds[var])
        # ds[var] = xr.where((ds[var] > 120) & (ds[var] <= 151), 5,  ds[var])
        # ds[var] = xr.where((ds[var] > 151) & (ds[var] <= 181), 6,  ds[var])
        # ds[var] = xr.where((ds[var] > 181) & (ds[var] <= 212), 7,  ds[var])
        # ds[var] = xr.where((ds[var] > 212) & (ds[var] <= 243), 8,  ds[var])
        # ds[var] = xr.where((ds[var] > 243) & (ds[var] <= 273), 9,  ds[var])
        # ds[var] = xr.where((ds[var] > 273) & (ds[var] <= 304), 10, ds[var])
        # ds[var] = xr.where((ds[var] > 304) & (ds[var] <= 334), 11, ds[var])
        # ds[var] = xr.where((ds[var] > 334) & (ds[var] <= 365), 12, ds[var])      

In [ ]:
# calculate phenometrics!
out = phenolopy.get_phenometrics(ds=ds, 
                                 metrics=None, # all
                                 #method='relative_amplitude', 
                                 method='seasonal_amplitude',
                                 factor=0.75, 
                                 #abs_value=0,
                                 peak_spacing=12,
                                 fix_edges=True,
                                 fill_nan=True)

for var in out:        
    if '_t' in var or 'roi' in var or 'rod' in var:
        cmap = 'jet'
    elif 'sos_t' in var or 'eos_t' in var:
        cmap = 'gist_rainbow'
    elif '_v' in var:
        cmap = 'RdYlGn'

    print(var)
    out[var].plot(robust=True, cmap=cmap)
    plt.show()
    plt.close()

In [ ]:
raise

In [ ]:
arr = ds['veg_idx'].isel(x=10, y=10).values
doy = ds['time.dayofyear']

plt.plot(arr)
plt.show()

In [ ]:
rra = np.flip(arr)
doy = ds['time.dayofyear']

plt.plot(rra)
plt.show()

In [ ]:
idx

In [ ]:
def _pos(arr, doy, fix_edges):

    # mask extreme edges
    if fix_edges:
        arr = phenolopy.clean_edges(arr, doy, 'max')
    
    #if doy[np.nanargmax(arr)] == 1:
        #max_idxs = (-arr).argsort()[:365]
        #idx = max_idxs[max_idxs > 31][0]
    #elif doy[np.nanargmax(arr)] == 1:
        #max_idxs = (-arr).argsort()[:365]
        #idx = max_idxs[max_idxs < 334][0]
    #else:
        #idx = np.nanargmax(arr)

    # get max value index
    idx = np.nanargmax(arr)

    # set values
    v, t = arr[idx], doy[idx]

    return v, t

# calculate peak of season (pos) values and times (doys)
da_pos_v, da_pos_t = xr.apply_ufunc(_pos,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'fix_edges': fix_edges})

da_pos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_pos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _vos(arr, doy, fix_edges):
    
    # mask extreme edges
    if fix_edges:
        arr = phenolopy.clean_edges(arr, doy, 'min')

    # get min value index
    idx = np.nanargmin(arr)

    # set values
    v, t = arr[idx], doy[idx]

    return v, t  

# calculate valley of season (vos) values and times (doys)
da_vos_v, da_vos_t = xr.apply_ufunc(_vos,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'fix_edges': fix_edges})

da_vos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_vos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _mos(arr, doy, fix_edges):
        
    # mask extreme edges
    if fix_edges:
        arr = phenolopy.clean_edges(arr, doy, 'max')
    
    # get pos time (doy)
    pos_t = doy[np.nanargmax(arr)]

    # get slope values left, right of pos time (doy)
    s_l, s_r = arr[doy <= pos_t], arr[doy >= pos_t]

    # get upper 80% values on left, right slopes
    s_l = s_l[s_l >= np.nanpercentile(s_l, 80)]
    s_r = s_r[s_r >= np.nanpercentile(s_r, 80)]

    # get mean of left, right slope
    s_l, s_r = np.nanmean(s_l), np.nanmean(s_r)

    # get mean of both
    v = np.nanmean([s_l, s_r])

    return v

# calculate middle of season (mos) values (no times)
da_mos_v = xr.apply_ufunc(_mos,
                          ds['veg_idx'],
                          ds['time.dayofyear'],
                          input_core_dims=[['time'], ['time']],
                          output_core_dims=[[]],
                          vectorize=True,
                          dask='allowed',
                          output_dtypes=['float32'],
                          kwargs={'fix_edges': fix_edges})

da_mos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
def _bse(arr, doy, fix_edges):
    
    # mask extreme edges
    if fix_edges:
        arr = phenolopy.clean_edges(arr, doy, 'min')
    
    # get pos time (doy)
    pos_t = doy[np.nanargmax(arr)]

    # get slope values left, right of pos time (doy)
    s_l, s_r = arr[doy <= pos_t], arr[doy >= pos_t]
    
    # get min value on left, right of slope
    s_l, s_r = np.nanmin(s_l), np.nanmin(s_r)

    # get mean of both
    v = np.nanmean([s_l, s_r])

    return v

# calculate base (bse) values (no times)
da_bse_v = xr.apply_ufunc(_bse,
                          ds['veg_idx'],
                          ds['time.dayofyear'],
                          input_core_dims=[['time'], ['time']],
                          output_core_dims=[[]],
                          vectorize=True,
                          dask='allowed',
                          output_dtypes=['float32'],
                          kwargs={'fix_edges': fix_edges})

da_bse_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
def _aos(arr_upper, arr_lower):
    
    # get aos values
    v = np.abs(arr_upper - arr_lower)

    return v

# calculate amplitude of season (aos) values (no times)
da_aos_v = xr.apply_ufunc(_aos,
                          da_mos_v,
                          da_bse_v,
                          dask='allowed',
                          output_dtypes=['float32'])

da_aos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
def _sos_fos(arr, doy, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')       
        
        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values left of pos and where slope +
        diff = np.where(np.gradient(arr) > 0, True, False)
        s_l = np.where((doy <= pos_t) & diff, arr, np.nan)

        # build non-nan start and end index ranges for slope +
        clusts = np.ma.clump_unmasked(np.ma.masked_invalid(s_l))

        # if cluster(s) exist, get last element of slope, else last element
        idx = 0
        if len(clusts) != 0:
            idx = clusts[0].start

        # set values
        v, t = arr[idx], doy[idx]
    
    except:
        v, t = arr_tmp[0], doy[0]

    return v, t  

# calculate start of season (sos) values and times (doys)
da_sos_v, da_sos_t = xr.apply_ufunc(_sos_fos,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'fix_edges': fix_edges})

da_sos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_sos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _eos_fos(arr, doy, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')
        
        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values right of pos and where slope -
        diff = np.where(np.gradient(arr) < 0, True, False)
        s_r = np.where((doy >= pos_t) & diff, arr, np.nan)

        # build non-nan start and end index ranges for slope -
        clusts = np.ma.clump_unmasked(np.ma.masked_invalid(s_r))

        # if cluster(s) exist, get last of slope
        idx = -1
        if len(clusts) != 0:
            idx = clusts[0].stop - 1
        
        # set values
        v, t = arr[idx], doy[idx]
    
    except:
        v, t = arr_tmp[-1], doy[-1]

    return v, t   

# calculate end of season (eos) values and times (doys)
da_eos_v, da_eos_t = xr.apply_ufunc(_eos_fos,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'fix_edges': fix_edges})

da_eos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_eos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _sos_mos(arr, doy, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')
        
        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values left of pos and where slope +
        diff = np.where(np.gradient(arr) > 0, True, False)
        
        s_l = np.where((doy <= pos_t) & diff, arr, np.nan)
        
        # get mean of all values in slope + areas
        mean = np.nanmean(s_l)
        
        # calc abs distances of each slope + val and mean
        dists = np.abs(s_l - mean)
        
        # get the smallest idx
        idx = np.nanargmin(dists)
        
        # set values
        v = arr[idx]
        t = doy[idx]
    
    except:
        v, t = arr_tmp[0], doy[0]

    return v, t  

# calculate start of season (sos) values and times (doys)
da_sos_v, da_sos_t = xr.apply_ufunc(_sos_mos,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'fix_edges': fix_edges})

da_sos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_sos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _eos_mos(arr, doy, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values right of pos and where slope -
        diff = np.where(np.gradient(arr) < 0, True, False)
        s_r = np.where((doy >= pos_t) & diff, arr, np.nan)
        
        # get mean of all values in slope - areas
        mean = np.nanmean(s_r)
        
        # calc abs distances of each slope - val and mean
        dists = np.abs(s_r - mean)
        
        # get the smallest idx
        idx = np.nanargmin(dists)
        
        # set values
        v = arr[idx]
        t = doy[idx]

    except:
        v, t = arr_tmp[-1], doy[-1]
        
    return v, t

# calculate end of season (eos) values and times (doys)
da_eos_v, da_eos_t = xr.apply_ufunc(_eos_mos,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'fix_edges': fix_edges})

da_eos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_eos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _sos_seaamp(arr, doy, arr_aos_v, arr_bse_v, factor, fix_edges):   
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values left of pos and where slope +
        diff = np.where(np.gradient(arr) > 0, True, False)       
        s_l = np.where((doy <= pos_t) & diff, arr, np.nan)

        # add base to amplitude and threshold it via factor
        samp = float((arr_aos_v * factor) + arr_bse_v)

        # calc abs distances of each slope + val to seas amp
        dists = np.abs(s_l - samp)

        # get the smallest idx
        idx = np.nanargmin(dists)

        # set values
        v, t = arr[idx], doy[idx]
        
    except:
        v, t = arr_tmp[0], doy[0]

    return v, t  

# calculate start of season (sos) values and times (doys)
da_sos_v, da_sos_t = xr.apply_ufunc(_sos_seaamp,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    da_aos_v,
                                    da_bse_v,
                                    input_core_dims=[['time'], ['time'], [], []],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'factor': 0.75, 'fix_edges': fix_edges})

da_sos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_sos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _eos_seaamp(arr, doy, arr_aos_v, arr_bse_v, factor, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values right of pos and where slope -
        diff = np.where(np.gradient(arr) < 0, True, False)       
        s_r = np.where((doy >= pos_t) & diff, arr, np.nan)

        # add base to amplitude and threshold it via factor
        samp = float((arr_aos_v * factor) + arr_bse_v)

        # calc abs distances of each slope - val to seas amp
        dists = np.abs(s_r - samp)

        # get the smallest idx
        idx = np.nanargmin(dists)

        # set values
        v, t = arr[idx], doy[idx]
        
    except:
        v, t = arr_tmp[-1], doy[-1]

    return v, t  

# calculate end of season (sos) values and times (doys)
da_eos_v, da_eos_t = xr.apply_ufunc(_eos_seaamp,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    da_aos_v,
                                    da_bse_v,
                                    input_core_dims=[['time'], ['time'], [], []],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'factor': 0.75, 'fix_edges': fix_edges})

da_eos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_eos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _sos_absamp(arr, doy, abs_value, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values left of pos and where slope +
        diff = np.where(np.gradient(arr) > 0, True, False)       
        s_l = np.where((doy <= pos_t) & diff, arr, np.nan)

        # calc abs distances of each slope + val to absolute val
        dists = np.abs(s_l - abs_value)

        # get the smallest idx
        idx = np.nanargmin(dists)

        # set values
        v, t = arr[idx], doy[idx]
        
    except:
        v, t = arr_tmp[0], doy[0]

    return v, t  

# calculate start of season (sos) values and times (doys)
da_sos_v, da_sos_t = xr.apply_ufunc(_sos_absamp,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'abs_value': 0.3, 'fix_edges': fix_edges})

da_sos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_sos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _eos_absamp(arr, doy, abs_value, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values right of pos and where slope -
        diff = np.where(np.gradient(arr) < 0, True, False)       
        s_r = np.where((doy >= pos_t) & diff, arr, np.nan)

        # calc abs distances of each slope - val to absolute val
        dists = np.abs(s_r - abs_value)

        # get the smallest idx
        idx = np.nanargmin(dists)

        # set values
        v, t = arr[idx], doy[idx]
        
    except:
        v, t = arr_tmp[-1], doy[-1]

    return v, t  

# calculate end of season (sos) values and times (doys)
da_eos_v, da_eos_t = xr.apply_ufunc(_eos_absamp,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'abs_value': 0.3, 'fix_edges': fix_edges})

da_eos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_eos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _sos_relamp(arr, doy, factor, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()

        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values left of pos and where slope +
        diff = np.where(np.gradient(arr) > 0, True, False)       
        s_l = np.where((doy <= pos_t) & diff, arr, np.nan)

        # get relative amp via robust max and base (10% cut off)
        q_min = np.nanpercentile(arr, q=10)
        q_max = np.nanpercentile(arr, q=90)
        ramp = float(q_max - q_min)

        # add max to relative amp and threshold it via factor
        ramp = (ramp * factor) + q_min
               
        # calc abs distances of each slope + val to relative value
        dists = np.abs(s_l - ramp)

        # get the smallest idx
        idx = np.nanargmin(dists)

        # set values
        v, t = arr[idx], doy[idx]

    except:
        v, t = arr_tmp[0], doy[0]

    return v, t  

# calculate start of season (sos) values and times (doys)
da_sos_v, da_sos_t = xr.apply_ufunc(_sos_relamp,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'factor': 0.75, 'fix_edges': fix_edges})

da_sos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_sos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _eos_relamp(arr, doy, factor, fix_edges):
    try:
        # make copy of arr for error handling
        arr_tmp = arr.copy()
        
        # mask extreme edges
        if fix_edges:
            arr = phenolopy.clean_edges(arr, doy, 'max')

        # get pos time (doy)
        pos_t = doy[np.nanargmax(arr)]

        # get all values right of pos and where slope +
        diff = np.where(np.gradient(arr) < 0, True, False)       
        s_r = np.where((doy >= pos_t) & diff, arr, np.nan)

        # get relative amp via robust max and base (10% cut off)
        q_min = np.nanpercentile(arr, q=10)
        q_max = np.nanpercentile(arr, q=90)
        ramp = float(q_max - q_min)

        # add max to relative amp and threshold it via factor
        ramp = (ramp * factor) + q_min

        # calc abs distances of each slope + val to relative value
        dists = np.abs(s_r - ramp)

        # get the smallest idx
        idx = np.nanargmin(dists)

        # set values
        v, t = arr[idx], doy[idx]
        
    except:
        v, t = arr_tmp[-1], doy[-1]

    return v, t  

# calculate end of season (eos) values and times (doys)
da_eos_v, da_eos_t = xr.apply_ufunc(_eos_relamp,
                                    ds['veg_idx'],
                                    ds['time.dayofyear'],
                                    input_core_dims=[['time'], ['time']],
                                    output_core_dims=[[], []],
                                    vectorize=True,
                                    dask='allowed',
                                    output_dtypes=['float32', 'float32'],
                                    kwargs={'factor': 0.75, 'fix_edges': fix_edges})

da_eos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

da_eos_t.plot(cmap='jet')
plt.show()

In [ ]:
def _los(arr_sos_t, arr_eos_t):

    # get los values
    v = np.abs(arr_eos_t - arr_sos_t)    

    return v

# calculate amplitude of season (aos) values (no times)
da_los_v = xr.apply_ufunc(_los,
                          da_sos_t,
                          da_eos_t,
                          vectorize=True,
                          dask='allowed',
                          output_dtypes=['float32'])

try:
    # interpolate nan values
    mask = xr.where(da_los_v.isnull(), True, False)
    da_roi_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                              da_los_v,
                              mask,
                              dask='allowed',
                              output_dtypes=['float32'],
                              kwargs={'method': 'nearest'})
except:
    pass

da_los_v.plot(cmap='jet', robust=True)
plt.show()

In [ ]:
def _roi(arr_pos_v, arr_pos_t, arr_sos_v, arr_sos_t):
    try:
        # get roi values   
        v = ((arr_pos_v - arr_sos_v) / 
             (arr_pos_t - arr_sos_t) * 10000)

        # get absolute if negatives
        v = np.abs(v)
    
    except:
        v = np.nan
    
    return v

# calculate rate of increase values (no times)
da_roi_v = xr.apply_ufunc(_roi,
                          da_pos_v,
                          da_pos_t,
                          da_sos_v,
                          da_sos_t,
                          #vectorize=True,
                          dask='allowed',
                          output_dtypes=['float32'])

try:
    # interpolate nan values
    mask = xr.where(da_roi_v.isnull(), True, False)
    da_roi_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                              da_roi_v,
                              mask,
                              dask='allowed',
                              output_dtypes=['float32'],
                              kwargs={'method': 'nearest'})
except:
    pass

da_roi_v.plot(cmap='jet', robust=True)
plt.show()

In [ ]:
def _rod(arr_pos_v, arr_pos_t, arr_eos_v, arr_eos_t):
    try:
        # get rod values   
        v = ((arr_eos_v - arr_pos_v) / 
             (arr_eos_t - arr_pos_t) * 10000)
        
        # get absolute if negatives
        v = np.abs(v)
        
    except:
        v = np.nan
    
    return v

# calculate rate of increase values (no times)
da_rod_v = xr.apply_ufunc(_rod,
                          da_pos_v,
                          da_pos_t,
                          da_eos_v,
                          da_eos_t,
                          input_core_dims=[[], [], [], []],
                          output_core_dims=[[]],
                          vectorize=True,
                          dask='allowed',
                          output_dtypes=['float32'])

try:
    # interpolate nan values
    mask = xr.where(da_rod_v.isnull(), True, False)
    da_rod_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                              da_rod_v,
                              mask,
                              dask='allowed',
                              output_dtypes=['float32'],
                              kwargs={'method': 'nearest'})
except:
    pass

da_rod_v.plot(cmap='jet', robust=True)
plt.show()

In [ ]:
def _sios(arr, doy, arr_sos_t, arr_eos_t, arr_bse_v):
    try:        
        # subtract the base from the original array
        arr = arr - arr_bse_v 
        
        # get all idxs between start and end of season
        season_idxs = np.where((doy >= arr_sos_t) & 
                               (doy <= arr_eos_t))

        # subset arr and doy whereever in range
        arr = arr[season_idxs]
        doy = doy[season_idxs]
        
        # get short integral of all values via trapz   
        v = np.trapz(y=arr, x=doy)
        
        # rescale if negatives
        #if np.min(v) < 0:
            #v += np.abs(np.nanmin(v))
    
    except:
        v = np.nan
    
    return v

# calculate short integral of season (no times)
da_sios_v = xr.apply_ufunc(_sios,
                           ds['veg_idx'],
                           ds['time.dayofyear'],
                           #da_sos_t,
                           #da_eos_t,
                           da_sos_t.rolling(x=3, y=3, center=True, min_periods=1).mean(),
                           da_eos_t.rolling(x=3, y=3, center=True, min_periods=1).mean(),
                           da_bse_v,
                           input_core_dims=[['time'], ['time'], [], [], []],
                           output_core_dims=[[]],
                           vectorize=True,
                           dask='allowed',
                           output_dtypes=['float32'])

if da_sios_v.min() < 0:
    da_sios_v = da_sios_v + np.abs(da_sios_v.min())

da_sios_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
def _lios(arr, doy, arr_sos_t, arr_eos_t):
    try:        
        # get all idxs between start and end of season
        season_idxs = np.where((doy >= arr_sos_t) &
                               (doy <= arr_eos_t))

        # subset arr and doy whereever in range
        arr = arr[season_idxs]
        doy = doy[season_idxs]
        
        # get long integral of all values via trapz   
        v = np.trapz(y=arr, x=doy)
        
        # rescale if negatives
        #if np.min(v) < 0:
            #v += np.abs(np.nanmin(v))
    
    except:
        v = np.nan
    
    return v

# calculate long integral of season (no times)
da_lios_v = xr.apply_ufunc(_lios,
                           ds['veg_idx'],
                           ds['time.dayofyear'],
                           #da_sos_t,
                           #da_eos_t,
                           da_sos_t.rolling(x=3, y=3, center=True, min_periods=1).mean(),
                           da_eos_t.rolling(x=3, y=3, center=True, min_periods=1).mean(),
                           input_core_dims=[['time'], ['time'], [], []],
                           output_core_dims=[[]],
                           vectorize=True,
                           dask='allowed',
                           output_dtypes=['float32'])

try:
    # interpolate nan values
    mask = xr.where(da_lios_v.isnull(), True, False)
    da_lios_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                               da_lios_v,
                               mask,
                               dask='allowed',
                               output_dtypes=['float32'],
                               kwargs={'method': 'nearest'})
except:
    pass

if da_lios_v.min() < 0:
    da_lios_v = da_lios_v + np.abs(da_lios_v.min())

da_lios_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
def _siot(arr, doy, arr_bse_v):
    try:        
        # subtract the base from the orig array
        arr = arr - arr_bse_v
                
        # get long integral of all values via trapz first
        v = np.trapz(y=arr, x=doy) 
        
        # rescale if negatives
        #if np.min(v) < 0:
            #v += np.abs(np.nanmin(v))

    except:
        v = np.nan
    
    return v

# calculate short integral of total (no times)
da_siot_v = xr.apply_ufunc(_siot,
                           ds['veg_idx'],
                           ds['time.dayofyear'],
                           da_bse_v,
                           input_core_dims=[['time'], ['time'], []],
                           output_core_dims=[[]],
                           vectorize=True,
                           dask='allowed',
                           output_dtypes=['float32'])


try:
    # interpolate nan values
    mask = xr.where(da_siot_v.isnull(), True, False)
    da_siot_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                               da_siot_v,
                               mask,
                               dask='allowed',
                               output_dtypes=['float32'],
                               kwargs={'method': 'nearest'})
except:
    pass


if da_siot_v.min() < 0:
    da_siot_v = da_siot_v + abs(da_siot_v.min())

da_siot_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
def _liot(arr, doy):
    try:
        # get long integral of all values via trapz   
        v = np.trapz(y=arr, x=doy)
        
        # rescale if negatives
        #if np.min(v) < 0:
            #v += np.abs(np.nanmin(v))
    
    except:
        v = np.nan
    
    return v

# calculate long integral of total (no times)
da_liot_v = xr.apply_ufunc(_liot,
                           ds['veg_idx'],
                           ds['time.dayofyear'],
                           input_core_dims=[['time'], ['time']],
                           output_core_dims=[[]],
                           vectorize=True,
                           dask='allowed',
                           output_dtypes=['float32'])


try:
    # interpolate nan values
    mask = xr.where(da_liot_v.isnull(), True, False)
    da_liot_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                              da_liot_v,
                              mask,
                              dask='allowed',
                              output_dtypes=['float32'],
                              kwargs={'method': 'nearest'})
except:
    pass

if da_liot_v.min() < 0:
    da_liot_v = da_liot_v + np.abs(da_liot_v.min())

da_liot_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
from scipy.signal import find_peaks
def _nos(arr, doy, peak_spacing):
    try:
        # calculate peak prominence, divide by months
        t = ((np.nanmax(arr) - np.nanmin(arr)) / 
             (len(doy) / peak_spacing))

        # generate peak information
        peaks, _ = find_peaks(arr, prominence=t)

        # set value to 1 season, else > 1
        v = len(peaks) if len(peaks) > 0 else 1

    except:
        v = np.nan
    
    return v

# calculate number of seasons (nos) (no times)
da_nos_v = xr.apply_ufunc(_nos,
                          ds['veg_idx'],
                          ds['time.dayofyear'],
                          input_core_dims=[['time'], ['time']],
                          output_core_dims=[[]],
                          vectorize=True,
                          dask='allowed',
                          output_dtypes=['float32'],
                          kwargs={'peak_spacing': 12})

try:
    # interpolate nan values
    mask = xr.where(da_nos_v.isnull(), True, False)
    da_nos_v = xr.apply_ufunc(phenolopy.interpolate_2d,
                              da_nos_v,
                              mask,
                              dask='allowed',
                              output_dtypes=['float32'],
                              kwargs={'method': 'nearest'})
except:
    pass

da_nos_v.plot(cmap='RdYlGn', robust=True)
plt.show()

In [ ]:
raise

In [ ]:
if doy[np.nanargmax(arr)] == 1:
    max_idxs = (-arr).argsort()[:365]
    idx = max_idxs[max_idxs > 31][0]
elif doy[np.nanargmax(arr)] == 1:
    max_idxs = (-arr).argsort()[:365]
    idx = max_idxs[max_idxs < 334][0]
else:
    idx = np.nanargmax(arr)


In [ ]:
# get all intersections with curve vs thresh line
tmp = np.full_like(arr, ramp)        
idxs = np.argwhere(np.diff(np.sign(tmp - arr))).flatten()
idxs = np.intersect1d(idxs, np.where(diff))
idx = idxs[0]

In [ ]:
def _x(arr, spike_factor):
        
    # get std cutoff
    cutoff = np.std(arr, axis=2) * spike_factor
    
    # get median
    win = sliding_window_view(arr, window_shape=3, axis=2)
    arr_med = np.nanmedian(win, axis=2)

    # prepend, append nan
    arr_med = np.insert(arr_med, 0, np.nan, axis=2)
    arr_med = np.append(arr_med, np.nan, axis=2)

    # calc abs difference between orig and med 
    #arr_dif = np.abs(arr - arr_med)

    # calc mask
    #arr_mask = arr_dif > cutoff

    # set to nan if rules broken at each masked value
    #for i in np.where(arr_mask)[0]:
        #v, l, r = arr[i], arr[i - 1], arr[i + 1]
        #_mean, _max = np.nanmean([l, r]), np.nanmax([l, r])

        #if v < (_mean - cutoff) or v > _max + cutoff:
            #arr[i] = np.nan

    return arr

# Quick Check

In [ ]:
    # set default params for tool
    inputs = {
        'in_nc': '',                              # input nc (i.e. temp nc)
        'out_nc': '',                             # output nc (i.e. t1a nc)
        'in_use_all_dates': True,                 # use all dates
        'in_specific_years': '2019;2020',         # specif year(s)
        'in_metrics': None,                       # metrics list
        'in_veg_idx': 'MAVI',                     # veg index name       
        'in_method_type': 'Seasonal amplitude',   # season method
        'in_amp_factor': 0.75,                    # amplitude factor
        'in_abs_value': 0.3,                      # absolute value
        'in_sav_win_length': 3,                # savitsky window size
        'in_sav_polyorder': 1,                    # savitsky poly order
        'in_out_win_length': 3,                # outlier window length
        'in_out_factor': 2,                       # outlier user factor
        'in_fix_edges': True,                     # fix doy edges
        'in_fill_nans': True,                     # fill empty pixels
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_add_result_to_map': True,             # add result to map
    }

In [66]:
def quick():
    in_nc = r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\dwer_1_ls_90_20_raw_odc.nc"
    out_nc = r'E:\Curtin\GDVII - General\Work Package 2\test_data\phenolopy_metrics\outputs\out_nc.nc'
        
    inputs = {
        'in_nc': in_nc, 
        'out_nc': out_nc, 
        'in_use_all_dates': True,
        'in_specific_years': '2019;2020', 
        'in_metrics': None,                       
        'in_veg_idx': 'MAVI',     
        'in_method_type': 'Seasonal amplitude',
        'in_amp_factor': 0.5, 
        'in_abs_value': 0.3, 
        'in_sav_win_length': 3, 
        'in_sav_polyorder': 1, 
        'in_out_win_length': 3, 
        'in_out_factor': 2,  
        'in_fix_edges': True,  
        'in_fill_nans': True, 
        'in_fmask_flags': 'Valid;Snow;Water', 
        'in_max_cloud': 10, 
        'in_add_result_to_map': True, 
    }
    
    arcpy.env.addOutputsToMap = True

    try:
        print('\nTest: Phenometrics defaults')
        with io.capture_output() as cap:
            arcpy.Phenolopy_Metrics_toolbox(**inputs)
    except Exception as e:
        print(e)
        
quick()


Test: Phenometrics defaults
